## Introduction
At the end of 2019, the coronavirus disease aroused in “Wuhan”, the city of china. Since January 2020, It has been extensively increased all around the world. On 30 January 2020, the WHO announced public health emergency internationally because of COVID-19. The disease was named coronavirus disease (COVID-19) by WHO in February 2020. The epidemic COVID-19 disease is caused by a virus called "Severe Acute Respiratory Syndrome" coronavirus-2 (SARS-CoV-2). The coronavirus is a family of several viruses that causes disease like "Severe Acute Respiratory Syndrome" (SARS-CoV) and "Middle East Respiratory Syndrome". Nowadays, countries like the USA, India, Brazil, and Russia struggling with the COVID-19 as it is the cause of the increase in the death rate. It can be spread to humans as well as other species of animals such as cats, cattle, bats, and camels. It is can be deadly for people with a weakened immune system. Physicians and specialists of infectious diseases around the world are starving hard to discover a treatment for the disease. The diseases can be spreading by touching and sitting or standing close to infected person. So, the COVID-19 infected person should to be identified and isolated before he might infect others. The earlier stage symptoms of COVID-19 include “cough”, “fatigue”, “fever”, and “myalgia”. The more intense situation of disease includes the heart damage, the respiratory problem, and internal infections. Some other abnormal states are observed in Covid-19 patients Chest CT and X-ray images. In most cases, the doctors use the patient’s X-ray and CT scan images of the patient’s chest for earlier stage diagnose of COVID-19. Chest radiography is the most widely used method of pneumonia diagnosis worldwide. It is a faster and cheaper clinical method of diagnosis. It is preferred over computed tomography (CT) and MRI because of lower radiation dose, easy availability, and cost. So, X-ray is the first choice of the radiologist for chest pathology detection and it is also applied for confirmation of COVID-19 patients. Therefore, this study focus on the use of X-ray images for COVID-19 detection in patients.

## Import libraries

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib
import pydicom as dicom
import cv2
import ast
import warnings
warnings.filterwarnings('ignore')

## Load data

In [ ]:
train_image = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
train_study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
sample_submission = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

In [ ]:
train_image.head()

In [ ]:
len(train_image)

In [ ]:
train_study.head()

In [ ]:
len(train_study)

In [ ]:
sample_submission.head()

In [ ]:
len(sample_submission)

## Merging train study in train

In [ ]:
train_study['StudyInstanceUID'] = train_study['id'].apply(lambda x: x.replace('_study', ''))
del train_study['id']
train_image = train_image.merge(train_study, on='StudyInstanceUID')
train_image.head()
def bar_plot(train_df, variable):
    var = train_df[variable]
    varValue = var.value_counts()
    
    # visualize
    plt.figure(figsize = (12,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n {}".format(variable,varValue))
    
train_image['target'] = 'Negative for Pneumonia'
train_image.loc[train_image['Typical Appearance']==1, 'target'] = 'Typical Appearance'
train_image.loc[train_image['Indeterminate Appearance']==1, 'target'] = 'Indeterminate Appearance'
train_image.loc[train_image['Atypical Appearance']==1, 'target'] = 'Atypical Appearance'
bar_plot(train_image, 'target') 

In [ ]:
train_image["target"].value_counts().head(7).plot(kind = 'pie', autopct='%1.1f%%', figsize=(8, 8)).legend()

In [ ]:
train_image.loc[0]

## Creating a function to get the image

In [ ]:
def extraction(i):
    path_train = '../input/siim-covid19-detection/' + 'train/' + train_image.loc[i, 'StudyInstanceUID']
    last_folder_in_path = os.listdir(path_train)[0]
    path_train = path_train + '/{}/'.format(last_folder_in_path)
    img_id = train_image.loc[i, 'id'].replace('_image','.dcm')
    print(img_id)
    data_file = dicom.dcmread(path_train+img_id)
    img = data_file.pixel_array
    return img

In [ ]:
image1=extraction(0)
image1

## Creating bounding boxes

In [ ]:
boxes = ast.literal_eval(train_image.loc[0, 'boxes'])
fig, ax = plt.subplots(1,1, figsize=(8,4))
for box in boxes:
    p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                      box['width'], box['height'],
                                      ec='r', fc='none', lw=1.5)
    ax.add_patch(p)
ax.imshow(image1,cmap='gray')
plt.show()

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(20,16))
fig.subplots_adjust(hspace=.1, wspace=.1)
axes = axes.ravel()

for row in range(9):
    img = extraction(row)
    # if (nan == nan)
    # False
    if (train_image.loc[row,'boxes'] == train_image.loc[row,'boxes']):
        boxes = ast.literal_eval(train_image.loc[row,'boxes'])
        for box in boxes:
            p = matplotlib.patches.Rectangle((box['x'], box['y']),
                                              box['width'], box['height'],
                                              ec='r', fc='none', lw=2.
                                            )
            axes[row].add_patch(p)
    
    axes[row].imshow(img, cmap='gray')
    axes[row].set_title(train_image.loc[row, 'label'].split(' ')[0])
    axes[row].set_xticklabels([])
    axes[row].set_yticklabels([])

## Future work: Model training

## Consider upvoting if you like my work.